## estrazione features da reti pretrainate

In [15]:
import numpy as np
import collections
import SimpleITK as sitk
from scipy.ndimage import zoom
import nrrd
import os,sys
import pandas as pd
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model


## Singola Slice

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

model = VGG19(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# funzione per creazione maschera
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array,mask_array):
    # ridimensionamento
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    roi_images = image_array[zstart-1:zstop+1,ystart:ystop,xstart:xstop].transpose((2,1,0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1],1], order=3)
    roi_images2 = np.array(roi_images1,dtype=float)    

    # Trova la slice più grossa (basato sui pixel attivi della maschera)
    slice_sums = np.sum(mask_array[zstart-1:zstop+1, ystart:ystop, xstart:xstop], axis=(1, 2))  # Somma dei pixel per slice
    largest_slice_index = np.argmax(slice_sums)  # Indice della slice con il massimo numero di pixel attivi

    # Estrai solo la slice più grossa
    largest_slice_image = roi_images2[:, :, largest_slice_index]

    print(largest_slice_image.shape)

    # preprocessing per resnet
    x = image.img_to_array(largest_slice_image)
    print(x.shape)
    x = np.repeat(x, 3, axis=-1)
    x = np.expand_dims(x, axis=0)
    print(x.shape)
    x = preprocess_input(x)
    #x = np.transpose(x, (3, 1, 2, 0))
    

    # estrazione features principali come feature map
    base_model_pool_features = model.predict(x)

    feature_map = base_model_pool_features[0]

    #print(feature_map)

    feature_map = feature_map.transpose((2,1,0))
    features = np.max(feature_map,-1)
    features = np.max(features,-1)
    deeplearningfeatures = collections.OrderedDict()
    for ind_,f_ in enumerate(features):
    	deeplearningfeatures[str(ind_)] = f_
         
    return deeplearningfeatures


# salvataggio features in un file
featureDict = {}
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

        
    for t in os.listdir(filename):

        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):

        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    
    deeplearningfeatures = featureextraction(readdatadicom,readdatanrrd) 

    result = deeplearningfeatures
    key = list(result.keys())
    key = key[0:]
        
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[s] = feature
    dictkey = key
    print(s)
    
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\VGG19_NUOVO.csv')
    



## Multiple Slice

In [16]:
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet50(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# funzione per creazione maschera
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array, mask_array, patient_id):
    # Ridimensionamento della ROI
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    roi_images = image_array[zstart-1:zstop+1, ystart:ystop, xstart:xstop].transpose((2, 1, 0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1], 1], order=3)
    roi_images2 = np.array(roi_images1, dtype=float)

    # Prepara tutte le slice come un batch
    batch_slices = []
    for slice_index in range(roi_images2.shape[2]):
        slice_image = roi_images2[:, :, slice_index]
        x = image.img_to_array(slice_image)
        x = np.repeat(x, 3, axis=-1)  # Ripeti il canale se è un'immagine a singolo canale
        batch_slices.append(x)

    # Converti tutte le slice in un batch di input per ResNet
    batch_slices = np.array(batch_slices)
    batch_slices = preprocess_input(batch_slices)

    # Estrazione feature map per tutto il batch
    base_model_pool_features = model.predict(batch_slices)

    # Inizializza lista per salvare tutte le features
    all_features = []

    # Estrai le feature map per ogni slice e salva con il numero del paziente e della slice
    for slice_index in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[slice_index]

        # Trasposizione e riduzione delle features
        print(feature_map.shape)
        feature_map = feature_map.transpose((2, 1, 0))
        print(feature_map.shape)
        features = np.max(feature_map, -1)
        print(features.shape)
        features = np.max(features, -1)
        print(features.shape)

        # Aggiungi le feature con il numero della slice e del paziente
        feature_entry = {'Patient': patient_id, 'Slice': slice_index}
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    # Estrai tutte le features per tutte le slice
    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)

    # Aggiungi le feature di tutte le slice per questo paziente
    all_feature_dicts.extend(patient_features)

# Crea il DataFrame con le feature di tutte le slice e pazienti
dataframe = pd.DataFrame(all_feature_dicts)

# Salva il DataFrame in un file CSV
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\RESNET50_ALL_SLICES_new.csv', index=False)
    



100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)
(2048,)
(7, 7, 2048)
(2048, 7, 7)
(2048, 7)


KeyboardInterrupt: 

## Slice con indici corretti

In [14]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

model = InceptionV3(weights='imagenet', include_top=False)

pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# funzione per creazione maschera
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)
        
def featureextraction(image_array, mask_array, patient_id):
    # Ridimensionamento della ROI
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    # Qui stiamo lavorando con l'array originale, quindi manteniamo gli indici originali delle slice
    original_slice_indices = np.arange(zstart, zstop)  # Manteniamo gli indici delle slice originali

    # Tagliare il volume solo sulle slice che contengono la placca
    roi_images = image_array[zstart:zstop, ystart:ystop, xstart:xstop].transpose((2, 1, 0))
    roi_images1 = zoom(roi_images, zoom=[224/roi_images.shape[0], 224/roi_images.shape[1], 1], order=3)
    roi_images2 = np.array(roi_images1, dtype=float)

    # Prepara tutte le slice come un batch
    batch_slices = []
    for slice_index in range(roi_images2.shape[2]):
        slice_image = roi_images2[:, :, slice_index]
        x = image.img_to_array(slice_image)
        x = np.repeat(x, 3, axis=-1)  # Ripeti il canale se è un'immagine a singolo canale
        batch_slices.append(x)

    # Converti tutte le slice in un batch di input per ResNet
    batch_slices = np.array(batch_slices)
    batch_slices = preprocess_input(batch_slices)

    # Estrazione feature map per tutto il batch
    base_model_pool_features = model.predict(batch_slices)

    # Inizializza lista per salvare tutte le features
    all_features = []

    placca_indices_3d = list(range(zstart, zstop + 1))


    # Usa solo le slice corrispondenti agli indici originali
    for i in range(base_model_pool_features.shape[0]):
        feature_map = base_model_pool_features[i]

        # Trasposizione e riduzione delle features
        feature_map = feature_map.transpose((2, 1, 0))
        features = np.max(feature_map, -1)
        features = np.max(features, -1)

        # Associa l'indice reale della slice (non sequenziale) dal volume originale
        feature_entry = {'Patient': patient_id, 'Slice': original_slice_indices[i]}  # Usa l'indice reale della slice
        for ind_, f_ in enumerate(features):
            feature_entry[f'Feature_{ind_}'] = f_

        all_features.append(feature_entry)

    return all_features, placca_indices_3d


all_feature_dicts = []

for s in os.listdir(pathdicom):
    print(f"Processing patient: {s}")
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    # Estrai tutte le features e gli indici delle slice in 3D
    patient_features, placca_indices_3d = featureextraction(readdatadicom, readdatanrrd, patient_id=s)

    # Aggiungi gli indici 3D alle features
    for feature_dict, slice_index_3d in zip(patient_features, placca_indices_3d):
        feature_dict['3D_Slice_Index'] = slice_index_3d  # Aggiungi l'indice originale 3D della slice

    # Aggiungi le feature di tutte le slice per questo paziente
    all_feature_dicts.extend(patient_features)

# Crea il DataFrame con le feature di tutte le slice e pazienti
dataframe = pd.DataFrame(all_feature_dicts)

# Salva il DataFrame in un file CSV
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\INCEPTION_ALL_SLICES_with_3D_indices.csv', index=False)

print("CSV con gli indici 3D salvato con successo.")
    



Processing patient: 100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Processing patient: 101
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 234ms/step
Processing patient: 102
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 413ms/step
Processing patient: 103
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step
Processing patient: 104
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step
Processing patient: 105
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step
Processing patient: 106
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 583ms/step
Processing patient: 107
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 322ms/step
Processing patient: 108
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step
Processing patient: 109
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 258ms/step
Processing patient: 110
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 430ms/step
Processing patient: 111
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step
Processing patient: 112
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step
Processing patient: 113
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
Processing patient: 114
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step
Processing patient: 115
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 310ms/

## 3D

In [12]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import nrrd
from torchvision_3d.models import ResNet3D
from torchvision_3d.models import VGG3D
from torchvision_3d.models import DenseNet3D
from scipy.ndimage import zoom

# Carica la ResNet50 3D pretrainata
model = DenseNet3D(type='densenet121', pretrained=True)
model = model.eval()  # Modalità inferenza

# Funzione per il cropping della maschera 3D
def maskcroppingbox(images_array):
    images_array_2 = np.argwhere(images_array)
    (zstart, ystart, xstart), (zstop, ystop, xstop) = images_array_2.min(axis=0), images_array_2.max(axis=0) + 1
    return (zstart, ystart, xstart), (zstop, ystop, xstop)

# Funzione per l'estrazione delle feature da un volume 3D
def featureextraction(image_array, mask_array, patient_id):
    # Estrarre solo le slice che contengono la maschera
    (zstart, ystart, xstart), (zstop, ystop, xstop) = maskcroppingbox(mask_array)
    roi_images = image_array[zstart:zstop, ystart:ystop, xstart:xstop]  # Volume 3D
    
    # Ridimensionare il volume alla dimensione richiesta (224x224xD, dove D varia)
    depth = roi_images.shape[0]  # La profondità originale
    print("Original depth:", depth)

    # Ridimensiona altezza e larghezza a 224, mantieni invariata la profondità
    roi_images_resized = zoom(roi_images, zoom=[1, 224/roi_images.shape[1], 224/roi_images.shape[2]], order=3)
    #print("Shape after resize:", roi_images_resized.shape)

    roi_images_resized = np.expand_dims(roi_images_resized, axis=0)  # Aggiungi dimensione batch
    roi_images_resized = np.repeat(roi_images_resized, 3, axis=0)  # Da (1, D, H, W) a (3, D, H, W)

    # Converti il volume 3D in tensor
    volume_tensor = torch.tensor(roi_images_resized, dtype=torch.float32)
    volume_tensor = volume_tensor.unsqueeze(0)  # (1, 1, D, H, W) - batch size 1, canale 1

    #print(volume_tensor.shape)

    # Passa il volume 3D attraverso la rete pre-addestrata
    with torch.no_grad():
        features = model(volume_tensor)

    #print(f"Features shape: {features.shape}")
    # Estrai le feature map finali (saranno 2048 features per ResNet50)
    features = features.squeeze().numpy()
    print(f"Features shape: {features.shape}")
    features = np.max(features, axis=3)  # Riduci lungo la profondità
    features = np.max(features, axis=2)  # Riduci lungo una dimensione spaziale
    features = np.max(features, axis=1)  # Riduci lungo l'altra dimensione spaziale
    #print(f"Features shape: {features.shape}")

    # Aggiungi le feature con il numero del paziente e la profondità
    feature_entry = {'Patient': patient_id}
    for ind_, f_ in enumerate(features):
        feature_entry[f'Feature_{ind_}'] = f_

    return feature_entry

# Percorsi ai dati
pathdicom = "\\Users\\bsbar\\Desktop\\pazienti_nrrd"
pathroi = "\\Users\\bsbar\\Desktop\\Tesi\\ROI"

# Inizializza lista per salvare tutte le feature
all_feature_dicts = []

# Ciclo sui pazienti
for s in os.listdir(pathdicom):
    print(s)
    filename = os.path.join(pathdicom, s)

    for t in os.listdir(filename):
        pathdicomnew = os.path.join(pathdicom, s, t)
        readdatadicom, header = nrrd.read(pathdicomnew, index_order='C')

    pathroinew = os.path.join(pathroi, s)
    for g in os.listdir(pathroinew):
        troi = os.path.join(pathroi, s, g)
        readdatanrrd, header2 = nrrd.read(troi, index_order='C')

    # Estrai tutte le features per il volume 3D
    patient_features = featureextraction(readdatadicom, readdatanrrd, patient_id=s)

    # Aggiungi le feature di tutte le slice per questo paziente
    all_feature_dicts.append(patient_features)

# Crea il DataFrame con le feature di tutte le slice e pazienti
dataframe = pd.DataFrame(all_feature_dicts)

# Salva il DataFrame in un file CSV
dataframe.to_csv('C:\\Users\\bsbar\\Desktop\\DENSENET121_3D.csv', index=False)



c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bsbar\anaconda3\envs\iml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\bsbar/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:07<00:00, 4.07MB/s]


100
Original depth: 38
Features shape: (1024, 38, 7, 7)
101
Original depth: 42
Features shape: (1024, 42, 7, 7)
102
Original depth: 54
Features shape: (1024, 54, 7, 7)
103
Original depth: 46
Features shape: (1024, 46, 7, 7)
104
Original depth: 45
Features shape: (1024, 45, 7, 7)
105
Original depth: 38
Features shape: (1024, 38, 7, 7)
106
Original depth: 30
Features shape: (1024, 30, 7, 7)
107
Original depth: 44
Features shape: (1024, 44, 7, 7)
108
Original depth: 27
Features shape: (1024, 27, 7, 7)
109
Original depth: 41
Features shape: (1024, 41, 7, 7)
110
Original depth: 49
Features shape: (1024, 49, 7, 7)
111
Original depth: 26
Features shape: (1024, 26, 7, 7)
112
Original depth: 46
Features shape: (1024, 46, 7, 7)
113
Original depth: 27
Features shape: (1024, 27, 7, 7)
114
Original depth: 37
Features shape: (1024, 37, 7, 7)
115
Original depth: 44
Features shape: (1024, 44, 7, 7)
116
Original depth: 41
Features shape: (1024, 41, 7, 7)
117
Original depth: 33
Features shape: (1024, 33